# This file shows the process of creating an EA using ARIEL 

### How does ARIEL EC module work?

The ARIEL EC (Evolutionary Computing) module works a bit different than other EAs (Evolutionary Algorithms). While other EAs represent the population as a simple list of individuals, here they the population is made as its own class, with a population being type of `list[Individuals]`. 

Similarly, in traditional EA architecture an individual is chosen for certain operators (for example parent selection) according to some criteria, put into a separate list and then given to the operator function. ARIEL works by giving individuals what we call `tags`. An individual has `tags` that can be toggled, which qualify it for any and all operations. The tag can be whether an individual can crossover or mutate in the future, but it can also show if it can enter the learning cycle.

The tags can be changed at all times, and default values for each tag can be given to more closely represent a traditional EA structure.

Additionally, ARIEL utilizes an SQL database to handle the variables and outputs of the code. This makes the code run faster, but it adds an extra step to the process.

This file demonstrates the process of initializing an EA class and running it for a simple problem, in our case, the Sphere function.

In [2]:
# Standard library
import random
from typing import Literal, cast

# Pretty little errors and progress bars
from rich.console import Console
from rich.traceback import install

# Third-party libraries
import numpy as np

# Local libraries
from ariel.ec.a000 import IntegerMutator
from ariel.ec.a001 import Individual
from ariel.ec.a005 import Crossover
from ariel.ec.a004 import EASettings, EAStep, EA, Population

# Library to show fitness landscape
# from fitness_plot import fitness_landscape_plot


#### Define fitness function

In [4]:
#### Define the fitness function
def Ackley(x):
    """source: https://www.sfu.ca/~ssurjano/ackley.html"""

    x = np.array(x)
    # Ackley function parameters
    a = 20
    b = 0.2
    c = 2 * np.pi
    dimension = len(x)

    # Individual terms
    term1 = -a * np.exp(-b * np.sqrt(sum(x**2) / dimension))
    term2 = -np.exp(sum(np.cos(c * xi) for xi in x) / dimension)
    return term1 + term2 + a + np.exp(1)

def evaluate_ind(ind: Individual) -> float:
    """Evaluate an individual by calculating its fitness using the Ackley function."""

    return Ackley(cast("list[float]", ind.genotype))

def evaluate_pop(population: Population) -> Population:
    """Evaluate a population by calculating the fitness of each individual."""
    for ind in population:
        if ind.requires_eval:
            ind.fitness = evaluate_ind(ind)
    return population

# fitness_landscape_plot()


#### Initialize the global constants

In [5]:
# A seed is optional, but it helps with reproducibility
SEED = None  # e.g., 42

# The database has a few handling modes
    # "delete" will delete the existing database
    # "halt" will stop the execution if a database already exists
DB_HANDLING_MODES = Literal["delete", "halt"]

# Initialize RNG
RNG = np.random.default_rng(SEED)

# Initialize rich console and traceback handler
install()
console = Console()

### Initialize the EASettings class. 

The EASettings class acts as the handles the database and other parameters  

In [6]:
# Set config
config = EASettings()
config.is_maximisation = False
config.db_handling = "delete"


#### And just like that we have everything we need to get started. Now all we need to do define our evolutionary operators

Keep in mind that all operators in ARIEL have to work with the `Individual` and `Population` classes. You could define your own operators from scratch, but using the built in ones is easier.

In [7]:
def create_individual(num_dims) -> Individual:
    ind = Individual()
    ind.genotype = cast("list[float]", np.random.normal(loc=0, 
                                                        scale=50, 
                                                        size=num_dims).tolist())
    return ind

def parent_selection(population: Population) -> Population:
    """Tournament selection"""
    
    # Shuffle population to avoid bias
    random.shuffle(population)

    # Tournament selection
    for idx in range(0, len(population) - 1, 2):
        ind_i = population[idx]
        ind_j = population[idx + 1]

        # Compare fitness values and update tags
        if ind_i.fitness > ind_j.fitness and config.is_maximisation:
            ind_i.tags = {"ps": True}
            ind_j.tags = {"ps": False}
        else:
            ind_i.tags = {"ps": False}
            ind_j.tags = {"ps": True}
    return population
    
def crossover(population: Population) -> Population:
    """One point crossover"""

    parents = [ind for ind in population if ind.tags.get("ps", False)]
    for idx in range(0, len(parents), 2):
        parent_i = parents[idx]
        parent_j = parents[idx]
        genotype_i, genotype_j = Crossover.one_point(
            cast("list[float]", parent_i.genotype),
            cast("list[float]", parent_j.genotype),
        )

        # First child
        child_i = Individual()
        child_i.genotype = genotype_i
        child_i.tags = {"mut": True}
        child_i.requires_eval = True

        # Second child
        child_j = Individual()
        child_j.genotype = genotype_j
        child_j.tags = {"mut": True}
        child_j.requires_eval = True

        population.extend([child_i, child_j])
    return population

def mutation(population: Population) -> Population:
    for ind in population:
        if ind.tags.get("mut", False):
            genes = cast("list[int]", ind.genotype)
            mutated = IntegerMutator.float_creep(
                individual=genes,
                span=5,
                mutation_probability=0.5,
            )
            ind.genotype = mutated
    return population

def survivor_selection(population: Population) -> Population:

    # Shuffle population to avoid bias
    random.shuffle(population)
    current_pop_size = len(population)

    for idx in range(len(population)):
        ind_i = population[idx]
        ind_j = population[idx + 1]

        # Kill worse individual
        if ind_i.fitness > ind_j.fitness and config.is_maximisation:
            ind_j.alive = False
        else:
            ind_i.alive = False

        # Termination condition
        current_pop_size -= 1
        if current_pop_size <= config.target_population_size:
            break
    return population

### Define evolutionary loop

Now that all our operators are done, we can define the evolutionary loop and run the algorithm


In [8]:
def main() -> EA:
    """Entry point."""
    # Create initial population
    population_list = [create_individual(num_dims=10) for _ in range(10)]
    print(population_list)
    population_list = evaluate_pop(population_list)

    # Create EA steps
    ops = [
        EAStep("evaluation", evaluate_pop),
        EAStep("parent_selection", parent_selection),
        EAStep("crossover", crossover),
        EAStep("mutation", mutation),
        EAStep("evaluation", evaluate_pop),
        EAStep("survivor_selection", survivor_selection),
    ]

    # Initialize EA
    ea = EA(
        population_list,
        operations=ops,
        num_of_generations=100,
    )

    ea.run()

    best = ea.get_solution("best", only_alive=False)
    console.log(best)

    median = ea.get_solution("median", only_alive=False)
    console.log(median)

    worst = ea.get_solution("worst", only_alive=False)
    console.log(worst)

    return ea

In [9]:
ea = main()

[Individual(id=None, alive=True, time_of_birth=-1, time_of_death=-1, requires_eval=True, fitness_=None, requires_init=False, genotype_=[20.3165840779559, 94.55138342233215, -96.97248531665169, 51.9866373045062, -40.336023854890485, -86.14693661690809, -79.24702318025615, -23.599030201488784, -62.45758915901117, -64.22332864047893], tags_={}), Individual(id=None, alive=True, time_of_birth=-1, time_of_death=-1, requires_eval=True, fitness_=None, requires_init=False, genotype_=[-21.098101534896983, -45.7766371398363, -46.406411198180365, -79.49132052101143, 22.653475987776144, -10.162447053314215, -94.66378449483146, -32.31952872846253, 2.3230564983208466, -19.51246347631952], tags_={}), Individual(id=None, alive=True, time_of_birth=-1, time_of_death=-1, requires_eval=True, fitness_=None, requires_init=False, genotype_=[-25.192087066310613, -18.22795679192372, -21.210468109775267, -6.075354958849443, -22.37975567718726, -1.7028775668044556, -5.6288706226418554, -2.4327662913559567, 41.297

───────────────────────────────────────────────── EA Initialised ──────────────────────────────────────────────────

Output()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 ea = main()                                                                                  │
│   2                                                                                              │
│                                                                                                  │
│ in main:25                                                                                       │
│                                                                                                  │
│   22 │   │   num_of_generations=100,                                                             │
│   23 │   )                                                                                       │
│   24 │                                                                                           │
│ ❱ 25 │   ea.run()                                                                                │
│   26 │                                                                                           │
│   27 │   best = ea.get_solution("best", only_alive=False)                                        │
│   28 │   console.log(best)                                                                       │
│                                                                                                  │
│ /home/salo/projects/ariel-zoo/src/ariel/ec/a004.py:284 in run                                    │
│                                                                                                  │
│   281 │   │   │   range(self.num_of_generations),                                                │
│   282 │   │   │   description="Running EA:",                                                     │
│   283 │   │   ):                                                                                 │
│ ❱ 284 │   │   │   self.step()                                                                    │
│   285 │   │   self.console.rule("[green]EA Finished Running")                                    │
│   286                                                                                            │
│   287                                                                                            │
│                                                                                                  │
│ /home/salo/projects/ariel-zoo/src/ariel/ec/a004.py:276 in step                                   │
│                                                                                                  │
│   273 │   │   self.current_generation += 1                                                       │
│   274 │   │   self.fetch_population()                                                            │
│   275 │   │   for operation in self.operations:                                                  │
│ ❱ 276 │   │   │   self.population = operation(self.population)                                   │
│   277 │   │   self.commit_population()                                                           │
│   278 │                                                                                          │
│   279 │   def run(self) -> None:                                                                 │
│                                                                                                  │
│ /home/salo/projects/ariel-zoo/src/ariel/ec/a004.py:117 in __call__                               │
│                                                                                                  │
│   114 │   operation: Callable[[Population], Population]                                          │
│   115 │                                                    

## Work in progress 

### Will update with the plot showing the mean performance and standard deviation of the algorithm